In [94]:
# importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import calendar

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [81]:
df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0, low_memory=False)

In [82]:
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', '0'],
      dtype='object')

In [83]:
df = df.iloc[:119390]

In [84]:
# →  Estandarizar para que en todos los casos sean números y cambiar el datatype de la columna a número entero.

def cambiar_formato_mes(df):
    # Diccionario de mapeo de meses y valores numéricos
    dic_map = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 
        'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 
        'November': 11, 'December': 12, 
        1: 1, 2: 2, 3: 3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11, 12:12, 'nan': np.nan}
    
    # Asegurarse de que los valores en la columna sean de tipo string para mapear correctamente
    df['arrival_date_month'] = df['arrival_date_month'].astype(str).map(dic_map).astype('Int64')

    return df

df = cambiar_formato_mes(df)

In [85]:


def rellenar_fecha_llegada(df):  

    print(f'Nulos antes de hacer la operación:')
    print(f'- arrival_date_year: {df["arrival_date_year"].isna().sum()}')
    print(f'- arrival_date_month: {df["arrival_date_month"].isna().sum()}')
    print(f'- arrival_date_day_of_month: {df["arrival_date_day_of_month"].isna().sum()}')

    # Asegúrate de que las columnas de fecha estén en formato datetime
    df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'], errors='coerce')

    # Crear una columna para el total de la estancia
    df['total_stays'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

    # Inicializar la columna de fecha de llegada estimada con valores NaT (Not a Time)
    df['estimated_arrival_date'] = pd.NaT

    # Crear filtro para cuando el estado de la reserva es "Checkout"
    filtro_checkout = df['reservation_status'] == 'Checkout'

    # Calcular fecha de llegada estimada para "Checkout"
    df.loc[filtro_checkout, 'estimated_arrival_date'] = df.loc[filtro_checkout, 'reservation_status_date'] - pd.to_timedelta(df.loc[filtro_checkout, 'total_stays'], unit='D')

    # Para los que no son "Checkout", asumir que la llegada fue la fecha del estado de la reserva
    df.loc[~filtro_checkout, 'estimated_arrival_date'] = df.loc[~filtro_checkout, 'reservation_status_date']

    # Extraer el año, mes y día de la fecha de llegada estimada
    df['estimated_arrival_year'] = df['estimated_arrival_date'].dt.year
    df['estimated_arrival_month'] = df['estimated_arrival_date'].dt.month
    df['estimated_arrival_day'] = df['estimated_arrival_date'].dt.day

    # Rellenar los valores nulos en arrival_date_year, arrival_date_month y arrival_date_day_of_month con los valores estimados
    df['arrival_date_year'] = df['arrival_date_year'].fillna(df['estimated_arrival_year'])
    df['arrival_date_month'] = df['arrival_date_month'].fillna(df['estimated_arrival_month'])
    df['arrival_date_day_of_month'] = df['arrival_date_day_of_month'].fillna(df['estimated_arrival_day'])


    print(f'Nulos después de hacer la operación:')
    print(f'- arrival_date_year: {df["arrival_date_year"].isna().sum()}')
    print(f'- arrival_date_month: {df["arrival_date_month"].isna().sum()}')
    print(f'- arrival_date_day_of_month: {df["arrival_date_day_of_month"].isna().sum()}')

    # borramos las columnas creadas para hacer los cálculos
    columnas_a_borrar = ['estimated_arrival_date', 'estimated_arrival_year', 'estimated_arrival_month', 'estimated_arrival_day']
    df = df.drop(columns=columnas_a_borrar)

   


In [86]:
rellenar_fecha_llegada(df)

Nulos antes de hacer la operación:
- arrival_date_year: 54561
- arrival_date_month: 9390
- arrival_date_day_of_month: 119
Nulos después de hacer la operación:
- arrival_date_year: 5936
- arrival_date_month: 1042
- arrival_date_day_of_month: 13


In [87]:
def imputar_knn_fechas(df, n_neighbors=5):
    # Seleccionar las columnas relevantes para la imputación
    cols_fecha = ['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']
    
    # Filtrar el DataFrame para las columnas de fechas y convertirlas a float para KNNImputer
    df_fechas = df[cols_fecha].astype(float)
    
    # Imputar los valores nulos usando KNN
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputado = imputer.fit_transform(df_fechas)
    
    # Convertir el resultado de vuelta a un DataFrame con las mismas columnas
    df_imputado = pd.DataFrame(df_imputado, columns=cols_fecha)
    
    # Asegurarse de que los valores imputados sean enteros
    df_imputado = df_imputado.round().astype(int)
    
    # Verificar y ajustar los días para que no excedan los máximos permitidos por mes
    for index, row in df_imputado.iterrows():
        year = row['arrival_date_year']
        month = row['arrival_date_month']
        day = row['arrival_date_day_of_month']
        
        # Obtener el último día del mes específico
        last_day_of_month = calendar.monthrange(year, month)[1]
        
        # Ajustar si el día excede el máximo
        if day > last_day_of_month:
            df_imputado.at[index, 'arrival_date_day_of_month'] = last_day_of_month
    
    # Reemplazar las columnas originales en el DataFrame con los valores imputados y ajustados
    df[cols_fecha] = df_imputado
    
    # Imprimir los nulos restantes en las columnas relevantes
    print(f'Nulos después de la operación:')
    print(f'- arrival_date_year: {df["arrival_date_year"].isna().sum()}')
    print(f'- arrival_date_month: {df["arrival_date_month"].isna().sum()}')
    print(f'- arrival_date_day_of_month: {df["arrival_date_day_of_month"].isna().sum()}')



In [88]:
imputar_knn_fechas(df)

Nulos después de la operación:
- arrival_date_year: 0
- arrival_date_month: 0
- arrival_date_day_of_month: 0


In [89]:


def crear_columna_arrival_date(df):
    # Crear la columna de fecha combinada inicialmente
    df['arrival_date'] = pd.to_datetime(
        df[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].astype(str).agg('-'.join, axis=1), 
        format='%Y-%m-%d', 
        errors='coerce'  # Esto convertirá fechas inválidas a NaT
    )


In [90]:
crear_columna_arrival_date(df)

In [92]:

# Rellenar los valores nulos en 'arrival_date_week_number'con los valores calculados
df['arrival_date_week_number'] = df['arrival_date_week_number'].fillna(df['arrival_date'].dt.isocalendar().week)
print(f'- arrival_date_week_number: {df["arrival_date_week_number"].isna().sum()}')

- arrival_date_week_number: 0


In [95]:
def calcular_fecha_reserva(df):
    # Asegurar de que 'arrival_date' está en formato datetime
    df['arrival_date'] = pd.to_datetime(df['arrival_date'], errors='coerce')
    
    # Calcular 'reservation_date' restando 'lead_time' de 'arrival_date'
    df['reservation_date'] = df['arrival_date'] - pd.to_timedelta(df['lead_time'], unit='d')
    

In [96]:
calcular_fecha_reserva(df)

In [97]:
df["reservation_date"].isna().sum()

0

In [99]:
df.sample(20)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0,total_stays,estimated_arrival_date,estimated_arrival_year,estimated_arrival_month,estimated_arrival_day,arrival_date,reservation_date
98441,City Hotel,False,0.0,2016,9,40.0,29,0.0,1.0,2.0,0.0,0.0,SC,USA,NaN,TA/TO,0.0,NaN,0.0,A,A,0.0,152.0,NaN,0.0,Transient,106.65,0.0,0.0,Check-Out,2016-09-30,NaN,1.0,2016-09-30,2016.0,9.0,30.0,2016-09-29,2016-09-29
41884,City Hotel,False,2.0,2015,8,35.0,26,0.0,1.0,1.0,NaN,0.0,BB,PRT,NaN,Corporate,NaN,0.0,0.0,NaN,D,0.0,40.0,NaN,0.0,NaN,65.00,0.0,0.0,Check-Out,2015-08-27,NaN,1.0,2015-08-27,2015.0,8.0,27.0,2015-08-26,2015-08-24
23350,Resort Hotel,False,92.0,2016,4,17.0,21,0.0,2.0,1.0,0.0,0.0,BB,BEL,Online TA,TA/TO,0.0,NaN,0.0,A,A,0.0,240.0,NaN,0.0,NaN,49.00,0.0,2.0,Check-Out,2016-04-23,NaN,2.0,2016-04-23,2016.0,4.0,23.0,2016-04-21,2016-01-20
89954,City Hotel,False,12.0,2016,5,23.0,30,1.0,3.0,2.0,0.0,0.0,BB,NaN,Corporate,Corporate,0.0,0.0,0.0,NaN,A,1.0,NaN,NaN,0.0,NaN,111.35,0.0,0.0,Check-Out,2016-06-03,NaN,4.0,2016-06-03,2016.0,6.0,3.0,2016-05-30,2016-05-18
76513,City Hotel,True,195.0,2015,9,38.0,17,0.0,2.0,2.0,0.0,0.0,BB,NaN,NaN,TA/TO,0.0,1.0,0.0,A,A,0.0,20.0,NaN,0.0,NaN,48.00,0.0,0.0,Canceled,NaT,NaN,2.0,NaT,NaN,NaN,NaN,2015-09-17,2015-03-06
25173,Resort Hotel,False,7.0,2016,6,25.0,12,2.0,3.0,2.0,NaN,0.0,BB,FRA,NaN,TA/TO,0.0,NaN,0.0,NaN,G,0.0,240.0,NaN,0.0,Transient,189.00,0.0,0.0,Check-Out,NaT,NaN,5.0,NaT,NaN,NaN,NaN,2016-06-12,2016-06-05
97697,City Hotel,False,220.0,2015,9,38.0,17,2.0,1.0,1.0,0.0,0.0,HB,ITA,NaN,TA/TO,0.0,NaN,0.0,A,A,1.0,177.0,NaN,0.0,Transient-Party,128.00,0.0,0.0,Check-Out,NaT,NaN,3.0,NaT,NaN,NaN,NaN,2015-09-17,2015-02-09
52301,City Hotel,False,149.0,2016,6,23.0,3,0.0,1.0,2.0,0.0,0.0,BB,DEU,NaN,TA/TO,0.0,0.0,0.0,NaN,A,1.0,9.0,NaN,0.0,Transient,124.20,1.0,0.0,Check-Out,2016-06-04,NaN,1.0,2016-06-04,2016.0,6.0,4.0,2016-06-03,2016-01-06
36435,Resort Hotel,False,103.0,2017,5,20.0,14,2.0,5.0,2.0,NaN,0.0,BB,GBR,NaN,TA/TO,0.0,0.0,0.0,A,A,0.0,171.0,NaN,0.0,Transient,43.20,0.0,1.0,Check-Out,2017-05-21,NaN,7.0,2017-05-21,2017.0,5.0,21.0,2017-05-14,2017-01-31
52584,City Hotel,True,56.0,2016,6,23.0,8,0.0,1.0,2.0,NaN,0.0,BB,PRT,Offline TA/TO,Corporate,0.0,0.0,0.0,A,A,0.0,191.0,NaN,0.0,NaN,120.00,0.0,0.0,Canceled,2016-06-02,NaN,1.0,2016-06-02,2016.0,6.0,2.0,2016-06-08,2016-04-13
